<a href="https://colab.research.google.com/github/emguzzi/MasterThesisDemo/blob/main/SpeechCommands/TruncatedSignatureTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install signatory
!git clone https://github.com/emguzzi/MasterThesisDemo.git
import sys
sys.path.append('./MasterThesisDemo/PenDigits/')
from RandomSignature import *
import signatory
import torch
from sklearn.ensemble import RandomForestClassifier
from google.colab import drive
drive.mount("/content/drive")
import pickle
import numpy as np
from tqdm.auto import tqdm
from scipy.special import expit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.1 MB/s 
  Created wheel for signatory: filename=signatory-1.2.6.1.9.0-cp37-cp37m-linux_x86_64.whl size=7554437 sha256=f9a01b2cde1c6dd69f069b034bd8ae5e735adce0e4f61ffb2393e101eea9505a
  Stored in directory: /root/.cache/pip/wheels/12/ff/e5/ffe613433c810f7f82da6e0c55abd15f4cc04960f8137db53b
Successfully built signatory
Cloning into 'MasterThesisDemo'...
remote: Enumerating objects: 187, done.
remote: Counting objects: 100% (187/187), done.
remote: Compressing objects: 100% (161/161), done.
remote: Total 187 (delta 52), reused 130 (delta 20), pack-reused 0
Receiving objects: 100% (187/187), 9.07 MiB | 17.43 MiB/s, done.
Resolving deltas: 100% (52/52), done.
Mounted at /content/drive


In [2]:
## load the preprocessed paths
## data has to be stored on drive in a folder called SpeechCommands

with open('/content/drive/MyDrive/SpeechCommands/paths_time.pkl','rb') as f:
    paths = pickle.load(f)
with open('/content/drive/MyDrive/SpeechCommands/y_train.pkl','rb') as f:
    y_train = pickle.load(f)
with open('/content/drive/MyDrive/SpeechCommands/y_test.pkl','rb') as f:
    y_test = pickle.load(f)
with open('/content/drive/MyDrive/SpeechCommands/y_validation.pkl','rb') as f:
    y_validation = pickle.load(f)
    
paths_torch = torch.tensor(paths)
paths_np = np.array(paths)

In [3]:
## full truncated signature
truncated_sig = signatory.signature(paths_torch,2)

## consider the various level of the truncated signature separately
truncated_sigN1 = truncated_sig[:,:11]
truncated_sigN2 = truncated_sig[:,11:132]
truncated_sigN3 = truncated_sig[:,132:]


## subsample the features
omit_rate = 0.90
ind = np.random.choice(range(1463),replace = False,size = int(1463*(1-omit_rate)))
truncated_sig_subsampled = truncated_sig[:,ind]

## choose the features to train
#features = truncated_sig
features = truncated_sig_subsampled
#features = truncated_sigN3
#featrues = truncated_sigN2
#features = truncated_sigN1

##train and test
features_train = features[:27864,:] 
features_test = features[27864:31639,:]

clf = RandomForestClassifier(n_estimators = 1000, max_depth = 100)
clf.fit(features_train,y_train)
predictions = clf.predict(features_test)

print('Accuracy for the set of features considered: \n')
print(np.mean(predictions == y_test))


Accuracy for the set of features considered: 

0.6431788079470199


In [7]:
## consider a mix of the features of the truncated signature and the randomized signature
## compute randomized and truncated signature
def sigmoid(x):
    return 1/(1+np.exp(-x))

def identity(x):
    return x

hparams = {
'varA':1e-05, 
'mean':0,
'res_size':132, 
'activation': identity
}
# generate vector fields
[As,bs] = get_random_coeff(paths_np.shape[2],hparams)
# compute signature
rand_sig = compute_signature_vect(As,bs,paths_np,hparams)

## subsample the features
omit_rate = 0.90
ind = np.random.choice(range(132),replace = False,size = int(132*(1-omit_rate)))
rand_sig_subsampled = rand_sig[:,ind]

## choose the features to train
features = rand_sig
#features = rand_sig_subsampled

##train and test
features_train = features[:27864,:] 
features_test = features[27864:31639,:]

clf = RandomForestClassifier(n_estimators = 1000, max_depth = 100)
clf.fit(features_train,y_train)
predictions = clf.predict(features_test)

print('Accuracy for the set of features considered: \n')
print(np.mean(predictions == y_test))


  0%|          | 0/320 [00:00<?, ?it/s]

Accuracy for the set of features considered: 

0.4521854304635762


In [10]:
## consider a mix of randomized signature and truncated signature
#truncated_sig_rate = 0.5
#truncated_sig_rate_ind = np.random.choice(range(132),replace = False,size = int(132*truncated_sig_rate))
#rand_sig_ind = int(132-truncated_sig_rate)
#features = np.hstack([truncated_sig[:,truncated_sig_rate_ind],rand_sig[:,:rand_sig_ind]])

## add randomized signature to the truncated signature
features = np.hstack([truncated_sig,rand_sig])

##train and test
features_train = features[:27864,:] 
features_test = features[27864:31639,:]

clf = RandomForestClassifier(n_estimators = 1000, max_depth = 100)
clf.fit(features_train,y_train)
predictions = clf.predict(features_test)

print('Accuracy for the set of features considered: \n')
print(np.mean(predictions == y_test))

Accuracy for the set of features considered: 

0.7290066225165563
